In [94]:
%reset -f
import cv2
import glob
import random
import math
import numpy as np
import dlib
import itertools
import pickle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import sys


In [95]:
emotions = ["anger", "contempt", "disgust", "fear",
            "happiness", "neutral", "sadness", "surprise"]
emotions

['anger',
 'contempt',
 'disgust',
 'fear',
 'happiness',
 'neutral',
 'sadness',
 'surprise']

In [96]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #Для повышения контраста изображения применяется адаптивная эквализация 
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [97]:
# Установим классификатор метод опорных векторов с полиномиальным ядром
clf = SVC(kernel='linear')

In [98]:
# Сделать словарь для всех значений
data = {}

In [99]:
# Определить функцию для получения списка файлов,
# случайным образом перетасовать его и разделить на 80/20
def get_files(emotion):
    
    files = glob.glob("database\sorted_set\%s\*" % emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)]
    prediction = files[-int(len(files)*0.2):]
    
    return training, prediction

In [100]:
def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): # Для всех обнаруженных экземпляров лица в отдельности
        shape = predictor(image, d) # Нарисовать лич.точки с классом прогнозирования
        xlist = []
        ylist = []
        
        # Сохранить координаты X и Y в двух списках
        for i in range(0, 68):
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
            
        # Ищем сред.точки
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        
        # Ищем сред. точку на лице
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(w)
            landmarks_vectorised.append(z)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append((math.atan2(y, x)*360)/(2*math.pi))

        data['landmarks_vectorised'] = landmarks_vectorised
    if len(detections) < 1: 
        data['landmarks_vestorised'] = "error"

In [101]:
def make_sets():
    
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    
    for emotion in emotions:
        print(" Работаем с  %s" %emotion)
        training, prediction = get_files(emotion)
        
        # Добавим данные в список обучения и прогнозирования и сгенерируем метки 0-7
        for item in training:
            image = cv2.imread(item) # открываем изображение
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # конверт в серый цвет
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorised'] == "error":
                print("no face detected on this one")
            else:
                training_data.append(data['landmarks_vectorised']) # добавим массив изображений в обучающую выборку
                training_labels.append(emotions.index(emotion))
    
        for item in prediction:
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            get_landmarks(clahe_image)
            if data['landmarks_vectorised'] == "Ошибка":
                print("Не найдено лицо")
            else:
                prediction_data.append(data['landmarks_vectorised'])
                prediction_labels.append(emotions.index(emotion))
    
    return np.array(training_data), np.array(training_labels), \
           np.array(prediction_data), np.array(prediction_labels)

In [102]:
accur_lin = []
for i in range(0, 1): # сколько обучаться кругов будет
    
    # Получим из массива данные
    print("Круг %s" % i)
    X_train, y_train, X_test, y_test = make_sets()

    print("Обучение SVM linear %s" % i)
    clf.fit(X_train, y_train)

    # Делаем обучающую выборку для классификатора
    print("Для круга  %s" % i) 
    pred_lin = clf.score(X_test, y_test)
    
    print("Lenear: ", pred_lin)
    
    # Наша точность
    accur_lin.append(pred_lin)

# Получим среднюю точность работы
print("Среднее значениеe lin svm: %s" % np.mean(accur_lin))

Круг 0
 Работаем с  anger
 Работаем с  contempt
 Работаем с  disgust
 Работаем с  fear
 Работаем с  happiness
 Работаем с  neutral
 Работаем с  sadness
 Работаем с  surprise
Обучение SVM linear 0
Для круга  0
Lenear:  0.881818181818
Среднее значениеe lin svm: 0.881818181818


In [89]:
# save classifier
with open('svr.pickle', 'wb') as f:
  pickle.dump(clf, f)

In [90]:
# clf = pickle.load(open('svr.pickle', 'rb'))

In [91]:
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)

In [92]:
nmbr_of_test_examples = len(y_test)
cm_percent = cm / nmbr_of_test_examples
cm_percent

NameError: name 'cm' is not defined

In [93]:
plt.rcParams.update(
    {
        'font.size': 16,
#         'font.family': 'Times New Roman'
    })

plt.figure(figsize=(9, 9))
plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel('Предсказанный класс')
plt.ylabel('Настоящий класс')
for (i, j), z in np.ndenumerate(cm_percent):
    if i==4 and j==4:
        plt.text(j, i, '{:0.1f} %'.format(z), ha='center', va='center',
                 color='w')
    else:
        plt.text(j, i, '{:0.1f} %'.format(z), ha='center', va='center')
plt.show()

NameError: name 'cm' is not defined

In [75]:
def get_files(emotion):
    
    files = glob.glob("database\sorted_set\%s\*" % emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)]
    prediction = files[-int(len(files)*0.2):]
    
    return training, prediction

In [10]:
get_files(emotions[0])

(['database\\sorted_set\\anger\\11_004_00000021.png',
  'database\\sorted_set\\anger\\03_001_00000071.png',
  'database\\sorted_set\\anger\\11_006_00000010.png',
  'database\\sorted_set\\anger\\29_006_00000010.png',
  'database\\sorted_set\\anger\\42_004_00000020.png',
  'database\\sorted_set\\anger\\45_005_00000030.png',
  'database\\sorted_set\\anger\\26_008_00000029.png',
  'database\\sorted_set\\anger\\27_010_00000018.png',
  'database\\sorted_set\\anger\\99_001_00000018.png',
  'database\\sorted_set\\anger\\36_005_00000010.png',
  'database\\sorted_set\\anger\\00_005_00000023.png',
  'database\\sorted_set\\anger\\02_001_00000016.png',
  'database\\sorted_set\\anger\\92_003_00000014.png',
  'database\\sorted_set\\anger\\28_001_00000024.png',
  'database\\sorted_set\\anger\\17_006_00000010.png',
  'database\\sorted_set\\anger\\10_004_00000019.png',
  'database\\sorted_set\\anger\\30_007_00000020.png',
  'database\\sorted_set\\anger\\90_007_00000014.png',
  'database\\sorted_set\\ang

In [ ]:
cm_cnn = ([[6,0,0]])